# Idea: ABcache
## getting a precomputed ABcache file containing the parameters for beta-binomial distribution from the PoN-list

# Running EBfilter createCache on testdata

## setting the config

In [30]:
cd EBFilter/

/Users/martinscience/Sites/Python/EBFilter


In [2]:
from codes import run

# snakemake config
config = {'EB':{'run': True}}
params = {}
params['map_quality'] = 20
params['base_quality'] = 15
params['filter_flags'] = 'UNMAP,SECONDARY,QCFAIL,DUP'
params['fitting_penalty'] = .5
params['caching'] = True
# to simulate snakemake behavior
config['EB']['threads'] = 1
config['EB']['params'] = params
config['EB']
config['annovar'] = {'sep': '\t'}
config['EB']['log'] = 'output/logfile'


params = config['EB']['params']
threads = config['EB']['threads']
log = config['EB']['log']
sep = config['annovar']['sep']
_q = str(params['map_quality'])  # mapping quality=20
_Q = params['base_quality']      # base quality=15
fit_pen = params['fitting_penalty']
filter_quals = ''
for qual in range( 33, 33 + _Q ): 
    filter_quals += chr( qual )  # qual asciis for filtering out
_ff = params['filter_flags']     # 'UNMAP,SECONDARY,QCFAIL,DUP'
config = {'q':_q, 'Q':_Q, 'filter_quals': filter_quals, 'log':log, 'fitting_penalty': fit_pen, 'ff':_ff, 'threads':threads, 'sep': sep}


ModuleNotFoundError: No module named 'vcf'

## running makeEBcache on testdata

In [3]:
args = {}
args['pon_list'] = 'testdata/PoN_list.txt'
args['cache_folder'] = 'testdata/testdata_cache' # provide a folder for storing the data (snakemake config)
args['force_caching'] = True                 # force cache generation although no bed_file is provided
args['bed_file'] = None # 'testdata/input.bed'
args['generate_cache'] = True

In [ ]:
config['threads'] = 3
config['debug_mode'] = False
run.main(args, config)
!ls output

Generating Cache...
Generating  pileup for chromosome chr11..
Splitting bam files for chromosome chr11..
Writing pileup of Chr chr11 to file testdata/testdata_cache/cache_pileups/cache_chr11.pileup
Reading pileup testdata/testdata_cache/cache_pileups/cache_chr11.pileup for AB computation
Splitting the 457702 lines of chr11.pileup into 3 chunks for multithreaded computation..
Process 7973: Computing ABs for Chr chr11 	(lines 0	 to 	152568)
Process 7974: Computing ABs for Chr chr11 	(lines 152568	 to 	305135)
Process 7975: Computing ABs for Chr chr11 	(lines 305135	 to 	457702)


In [16]:
import pandas as pd
import sys
import os
def bed2chr_list(bed_file):
    '''
    generates a chromosome list from a bed file
    '''

    bed_df = pd.read_csv(bed_file, sep='\t', dtype={0:str}, header=None, skiprows=10)
    # return the list of unique values from the first row (Chr row)
    return bed_df.iloc[:,0].unique()

chr_list = bed2chr_list('testdata/input.anno')



In [15]:
chr_list

array(['chr11'], dtype=object)

### using CLI:

In [ ]:
!makeEBcache -t3 -force_caching testdata/PoN_list.txt testdata/testdata_cache

## running EBscore in cache_mode on testdata

In [ ]:
args = {}
args['mut_file'] = 'testdata/input.anno'
args['tumor_bam'] = 'testdata/tumor.bam'
args['pon_list'] = 'testdata/PoN_list.txt'
args['output_path'] = 'output/testdata_eb.csv'
args['use_cache'] = 'testdata/testdata_cache'
args['chromosome'] = 'chr11'
args['generate_cache'] = False

In [ ]:
config['threads'] = 20
config['debug_mode'] = False
run.main(args, config)
!ls output

### using CLI:

In [ ]:
!EBscore -t 3 -use_cache testdata/testdata_cache testdata/input.anno testdata/tumor.bam testdata/PoN_list.txt output/testdata_EB.csv

## Running EBfilter createCache on my data

In [ ]:
import os
HOME = os.environ['HOME'] # set HOME to run on different Macs
args = {}
args['pon_list'] = f'{HOME}/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/aml_pon.list'
args['cache_folder'] = f'{HOME}/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/aml_cache'
args['force_caching'] = True
args['bed_file'] = None
args['generate_cache'] = True

In [ ]:
config['threads'] = 3
config['debug_mode'] = True

### using CLI:

In [ ]:
!makeEBcache -t3 -force_caching ~/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/aml_pon.list ~/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/aml_cache


## Running EBscore in cache_mode on my data

In [ ]:
import os
HOME = os.environ['HOME'] # set HOME to run on different Macs
args = {}
args['mut_file'] = f'{HOME}/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/anno/test_rel.csv'
args['tumor_bam'] = f'{HOME}/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/bam/test_Rel1.bam'
args['pon_list'] = f'{HOME}/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/aml_pon.list'
args['use_cache'] = f'{HOME}/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/aml_cache'
args['output_path'] = f'{HOME}/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/output/test_eb.csv'
args['chromosome'] = 'chr11'
args['generate_cache'] = False

In [ ]:
config['threads'] = 40
config['debug_mode'] = True
run.main(args, config)

### using CLI:

In [ ]:
!EBscore -t 3 -use_cache ~/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/aml_cache ~//Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/anno/test_rel.csv ~/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/bam/test_Rel1.bam ~/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/aml_pon.list ~/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/output/test_eb.csv


## Running EBfilter createCache in BIHCluster on real data

In [ ]:
import os
HOME = os.environ['HOME'] # set HOME to run on different Macs
args = {}
args['pon_list'] = f'{HOME}/work/static/ref/PoN/AML_Pon.txt'
args['cache_folder'] = f'{HOME}/work/static/ref/PoN/AML_cache'
args['force_caching'] = False
args['chromosome'] = '22'
args['bed_file'] = f'{HOME}/work/static/ref/bed_files/SureSelect/hg19/SS_HAEv6r2_hg19_Padded_nochr.bed'
args['generate_cache'] = True


In [ ]:
config['threads'] = 40
config['debug_mode'] = True
run.main(args, config)

### using CLI:

In [ ]:
!makeEBcache -t3 -bed_file ~/work/static/ref/bed_files/SureSelect/hg19/SS_HAEv6r2_hg19_Padded_nochr.bed ~/work/static/ref/PoN/AML_Pon.txt ~/work/static/ref/PoN/AML_cache


## Running EBscore in cache_mode in BIHCluster on real data

In [ ]:
from codes import run

# snakemake config
config = {'EB':{'run': True}}
params = {}
params['map_quality'] = 20
params['base_quality'] = 15
params['filter_flags'] = 'UNMAP,SECONDARY,QCFAIL,DUP'
params['fitting_penalty'] = .5
params['caching'] = True
# to simulate snakemake behavior
config['EB']['threads'] = 1
config['EB']['params'] = params
config['EB']
config['annovar'] = {'sep': '\t'}
config['EB']['log'] = 'output/logfile'


params = config['EB']['params']
threads = config['EB']['threads']
log = config['EB']['log']
sep = config['annovar']['sep']
_q = str(params['map_quality'])  # mapping quality=20
_Q = params['base_quality']      # base quality=15
fit_pen = params['fitting_penalty']
filter_quals = ''
for qual in range( 33, 33 + _Q ): 
    filter_quals += chr( qual )  # qual asciis for filtering out
_ff = params['filter_flags']     # 'UNMAP,SECONDARY,QCFAIL,DUP'
config = {'q':_q, 'Q':_Q, 'filter_quals': filter_quals, 'log':log, 'fitting_penalty': fit_pen, 'ff':_ff, 'threads':threads, 'sep': sep}


import os
HOME = os.environ['HOME'] # set HOME to run on different Macs
args = {}
args['mut_file'] = 'testdata2/anno/test_D-CR1_bull.csv'
args['sep'] = ','
args['output_path'] = 'output/test_D_EB.csv'
args['tumor_bam'] = 'testdata2/bam/test_D.bam'
args['pon_list'] = f'{HOME}/work/static/ref/PoN/AML_Pon.txt'
args['use_cache'] = f'{HOME}/work/static/ref/PoN/AML_cache'
args['chromosome'] = '22'
args['generate_cache'] = False


config['threads'] = 40
config['debug_mode'] = False
run.main(args, config)

In [ ]:
config['threads'] = 40
config['debug_mode'] = False
run.main(args, config)

In [39]:
import os
HOME = os.environ['HOME'] # set HOME to run on different Macs
import pandas as pd
eb = pd.read_csv(f'{HOME}/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/anno/test_rel.csv', sep='\t')
pon_list = ['1', '7']
eb.query(f'Chrom not in {pon_list} and (Star == 1016777 or Ref == "G")')

,Chrom,Star,End,Ref,Alt,mut_type,somatic_status
2,11,1016627,1016627,G,T,snp,Germline
4,11,1016733,1016733,G,T,snp,Germline
6,11,1016777,1016777,G,A,snp,Germline
7,11,1016801,1016801,G,C,snp,Germline
11,11,1016853,1016853,G,C,snp,Germline
13,11,1016887,1016887,G,A,snp,Germline
16,11,1016934,1016934,G,A,snp,Germline
17,11,1016944,1016944,G,C,snp,Germline
18,11,1016954,1016954,G,C,snp,Germline
19,11,1016967,1016967,G,C,snp,Germline


In [26]:
pon = set([1,2,6])
set([1,2,6]).issubset(pon)

True

In [41]:
if 'd':
    print('Hello')

Hello


In [49]:
region = '11:234234'

In [50]:
region.split(':')[0]

'11'

In [51]:
[] + [1]

[1]

In [52]:
str(['hello'])

"['hello']"